## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-27-14-52-16 +0000,wapo,Live updates: Shooting reported at Annunciatio...,https://www.washingtonpost.com/nation/2025/08/...
1,2025-08-27-14-52-16 +0000,wapo,Annunciation Catholic School is a K-8 private ...,https://www.washingtonpost.com
2,2025-08-27-14-50-57 +0000,bbc,"NHS to lose out on new drugs, pharma firm warns",https://www.bbc.com/news/articles/c4gzk1p90q1o...
3,2025-08-27-14-48-14 +0000,startribune,Live: Minneapolis says shooter ‘contained’ as ...,https://www.startribune.com/minneapolis-shooti...
4,2025-08-27-14-44-39 +0000,nyt,Court Records Detail Antisemitic Attack That A...,https://www.nytimes.com/2025/08/27/world/austr...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2314/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
83,trump,77
17,new,19
218,gaza,17
381,cook,15
380,lisa,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
202,2025-08-27-00-11-00 +0000,wsj,President Trump’s attempt to fire Fed governor...,https://www.wsj.com/politics/policy/trumps-fed...,189
220,2025-08-26-22-42-00 +0000,wsj,President Trump’s attempt to fire Federal Rese...,https://www.wsj.com/economy/central-banking/un...,176
221,2025-08-26-22-33-58 +0000,nypost,Fed governor Lisa Cook hires Hunter Biden lawy...,https://nypost.com/2025/08/26/us-news/fed-gove...,163
267,2025-08-26-20-04-00 +0000,wsj,President Trump has told advisers that he want...,https://www.wsj.com/politics/policy/trump-weig...,155
326,2025-08-26-16-12-46 +0000,wapo,Federal Reserve governor Lisa Cook says she’ll...,https://www.washingtonpost.com/business/2025/0...,155


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
202,189,2025-08-27-00-11-00 +0000,wsj,President Trump’s attempt to fire Fed governor...,https://www.wsj.com/politics/policy/trumps-fed...
116,52,2025-08-27-09-28-18 +0000,nyt,Israel Links Deadly Hospital Attack in Gaza to...,https://www.nytimes.com/2025/08/26/world/middl...
190,51,2025-08-27-01-00-00 +0000,wsj,The first Black woman to serve as a Federal Re...,https://www.wsj.com/economy/central-banking/li...
233,49,2025-08-26-21-51-00 +0000,wsj,Canada’s manufacturers and wholesalers look to...,https://www.wsj.com/articles/canada-factory-wh...
112,48,2025-08-27-09-40-35 +0000,nypost,California 7-month-old boy Emmanuel Haro was l...,https://nypost.com/2025/08/27/us-news/californ...
24,43,2025-08-27-14-10-02 +0000,nypost,Three people suspected of carrying out covert ...,https://nypost.com/2025/08/27/world-news/denma...
29,43,2025-08-27-13-51-00 +0000,nypost,Trump administration plans to reclaim control ...,https://nypost.com/2025/08/27/us-news/trump-ad...
98,40,2025-08-27-10-00-00 +0000,nypost,Grassley insists to ‘Pod Force One’ that Biden...,https://nypost.com/2025/08/27/us-news/grassley...
56,37,2025-08-27-12-11-15 +0000,wapo,"Man gets 89 days in prison, for every day he f...",https://www.washingtonpost.com/nation/2025/08/...
1,35,2025-08-27-14-52-16 +0000,wapo,Annunciation Catholic School is a K-8 private ...,https://www.washingtonpost.com


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
